In [1]:
from PIL import Image
import os
from tqdm import tqdm
import numpy as np
from sklearn.metrics import accuracy_score

In [2]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, Convolution2D
from keras.layers import Activation, Dropout, Flatten, Dense, Input, BatchNormalization, ZeroPadding2D
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from keras import optimizers

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 1050 Ti (CNMeM is disabled, cuDNN 5110)


In [3]:
# Data augmentation
train_datagen = ImageDataGenerator(
          rotation_range=40, width_shift_range=0.2,
          height_shift_range=0.2, rescale=1./100,
          shear_range=0.2, zoom_range=0.2,
          horizontal_flip=True, fill_mode='nearest'
           )

validate_datagen = ImageDataGenerator(rescale=1./100)

In [4]:
train_data_dir = 'Data/Heatmaps/Train-Simple-Float-Gauss/TrainFloatGaussian/'
validation_data_dir = 'Data/Heatmaps/Train-Simple-Float-Gauss/TrainFloatGaussian-Val/'
nb_train_samples = 1005 + 1035 + 1117
nb_validation_samples = 112 + 115 + 124
epochs = 100
batch_size = 16


In [5]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(3, 256, 256)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))

model.add(Dropout(0.5))
model.add(Dense(3))
model.add(Activation('sigmoid'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [6]:
train_generator = train_datagen.flow_from_directory(
                    train_data_dir,
                    target_size=(256, 256),
                    batch_size=batch_size,
                    class_mode='categorical'
                    )

validation_generator = validate_datagen.flow_from_directory(
                            validation_data_dir,
                            target_size=(256, 256),
                            batch_size=batch_size,
                            class_mode='categorical'
                            )

Found 3157 images belonging to 3 classes.
Found 351 images belonging to 3 classes.


In [7]:
callback = ModelCheckpoint(filepath='Workshop-Paper/Vanilla-Gaussian/vanila-gauss-{epoch:03d}-{val_acc:.3f}', save_best_only=True, monitor='val_acc', verbose=1)

In [8]:
model.load_weights('Workshop-Paper/Vanilla-Gaussian/old/vanila-gauss-last-iteration')

In [8]:
history = model.fit_generator(train_generator, callbacks=[callback],
                    steps_per_epoch=nb_train_samples // batch_size,
                    epochs=epochs,
                    validation_data=validation_generator,
                    validation_steps=nb_validation_samples // batch_size
                   )

Epoch 1/100
197/197 [==============================] - 151s 767ms/step - loss: 1.0610 - acc: 0.4294 - val_loss: 1.0233 - val_acc: 0.4881
Epoch 2/100
197/197 [==============================] - 146s 743ms/step - loss: 1.0264 - acc: 0.4648 - val_loss: 1.0196 - val_acc: 0.4554
Epoch 3/100
197/197 [==============================] - 147s 747ms/step - loss: 1.0176 - acc: 0.4859 - val_loss: 1.0199 - val_acc: 0.4911
Epoch 4/100
197/197 [==============================] - 135s 686ms/step - loss: 1.0005 - acc: 0.5030 - val_loss: 1.0081 - val_acc: 0.5030
Epoch 5/100
197/197 [==============================] - 134s 682ms/step - loss: 1.0070 - acc: 0.4954 - val_loss: 0.9863 - val_acc: 0.4970
Epoch 6/100
197/197 [==============================] - 134s 679ms/step - loss: 1.0060 - acc: 0.4903 - val_loss: 0.9963 - val_acc: 0.5417
Epoch 7/100
197/197 [==============================] - 134s 681ms/step - loss: 0.9941 - acc: 0.4989 - val_loss: 1.0264 - val_acc: 0.4762
Epoch 8/100
197/197 [====================

KeyboardInterrupt: 

In [9]:
history = model.fit_generator(train_generator, callbacks=[callback],
                    steps_per_epoch=nb_train_samples // batch_size,
                    epochs=epochs - 18,
                    validation_data=validation_generator,
                    validation_steps=nb_validation_samples // batch_size
                   )

Epoch 1/82
197/197 [==============================] - 225s 1s/step - loss: 0.9724 - acc: 0.5322 - val_loss: 0.9782 - val_acc: 0.4792
Epoch 2/82
197/197 [==============================] - 169s 859ms/step - loss: 0.9806 - acc: 0.5195 - val_loss: 0.9744 - val_acc: 0.4732
Epoch 3/82
197/197 [==============================] - 145s 735ms/step - loss: 0.9771 - acc: 0.5147 - val_loss: 1.0005 - val_acc: 0.4405
Epoch 4/82
197/197 [==============================] - 169s 857ms/step - loss: 0.9775 - acc: 0.5122 - val_loss: 0.9391 - val_acc: 0.5179
Epoch 5/82
197/197 [==============================] - 162s 820ms/step - loss: 0.9745 - acc: 0.5107 - val_loss: 1.0012 - val_acc: 0.4821
Epoch 6/82
197/197 [==============================] - 159s 807ms/step - loss: 0.9730 - acc: 0.5157 - val_loss: 1.0141 - val_acc: 0.4732
Epoch 7/82
197/197 [==============================] - 176s 893ms/step - loss: 0.9648 - acc: 0.5183 - val_loss: 0.9450 - val_acc: 0.5446
Epoch 8/82
197/197 [==============================]

Epoch 33/82
197/197 [==============================] - 205s 1s/step - loss: 0.9517 - acc: 0.5363 - val_loss: 1.0053 - val_acc: 0.5208
Epoch 34/82
197/197 [==============================] - 204s 1s/step - loss: 0.9602 - acc: 0.5321 - val_loss: 0.9388 - val_acc: 0.5417
Epoch 35/82
197/197 [==============================] - 202s 1s/step - loss: 0.9573 - acc: 0.5324 - val_loss: 0.9994 - val_acc: 0.5238
Epoch 36/82
197/197 [==============================] - 200s 1s/step - loss: 0.9533 - acc: 0.5350 - val_loss: 0.9728 - val_acc: 0.5506
Epoch 37/82
197/197 [==============================] - 202s 1s/step - loss: 0.9596 - acc: 0.5325 - val_loss: 0.9761 - val_acc: 0.5208
Epoch 38/82
197/197 [==============================] - 212s 1s/step - loss: 0.9639 - acc: 0.5245 - val_loss: 0.9592 - val_acc: 0.5298
Epoch 39/82
197/197 [==============================] - 209s 1s/step - loss: 0.9522 - acc: 0.5276 - val_loss: 0.9544 - val_acc: 0.5357
Epoch 40/82
197/197 [==============================] - 208s 1s

Epoch 66/82
197/197 [==============================] - 202s 1s/step - loss: 0.9530 - acc: 0.5308 - val_loss: 0.9585 - val_acc: 0.5536
Epoch 67/82
197/197 [==============================] - 195s 988ms/step - loss: 0.9547 - acc: 0.5485 - val_loss: 1.0032 - val_acc: 0.5119
Epoch 68/82
197/197 [==============================] - 203s 1s/step - loss: 0.9476 - acc: 0.5343 - val_loss: 0.9800 - val_acc: 0.5149
Epoch 69/82
197/197 [==============================] - 205s 1s/step - loss: 0.9434 - acc: 0.5317 - val_loss: 1.0266 - val_acc: 0.5179
Epoch 70/82
197/197 [==============================] - 199s 1s/step - loss: 0.9460 - acc: 0.5562 - val_loss: 0.9766 - val_acc: 0.5089
Epoch 71/82
197/197 [==============================] - 205s 1s/step - loss: 0.9519 - acc: 0.5306 - val_loss: 0.9843 - val_acc: 0.4881
Epoch 72/82
197/197 [==============================] - 206s 1s/step - loss: 0.9487 - acc: 0.5411 - val_loss: 0.9805 - val_acc: 0.5208
Epoch 73/82
197/197 [==============================] - 208s

In [3]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(3, 256, 256)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))

model.add(Dropout(0.5))
model.add(Dense(3))
model.add(Activation('sigmoid'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [4]:
model.load_weights('Workshop-Paper/Vanilla-Gaussian/vanila-gauss-079-0.598')

In [5]:
test_data_gen = ImageDataGenerator(rescale=1.0/100)
test_data_stream = test_data_gen.flow_from_directory(
                    'Data/Heatmaps/Train-Simple-Float-Gauss/ValidateFloatGaussian/',
                    target_size=(256, 256),
                    class_mode='categorical',
                    shuffle=False,
                    batch_size=1
                    )

Found 1979 images belonging to 3 classes.


In [6]:
predictions = model.predict_generator(test_data_stream, 1979, verbose=1)

1979/1979 [==============================] - 64s 32ms/step


In [7]:
actual_classes = test_data_stream.classes

In [8]:
len(actual_classes)

1979

In [9]:
pred_classes = np.argmax(predictions, axis=1)

In [10]:
len(pred_classes)

1979

In [11]:
print ('Vanila Simple Gauss Validation Accuracy Score:', accuracy_score(pred_classes, actual_classes))

Vanila Simple Gauss Validation Accuracy Score: 0.514906518444


In [19]:
train_data_gen = ImageDataGenerator(rescale=1.0/100)
train_data_stream = train_data_gen.flow_from_directory(
                    'Data/Heatmaps/Train-Simple-Float-Gauss/TrainFloatGaussian/',
                    target_size=(256, 256),
                    class_mode='categorical',
                    shuffle=False,
                    batch_size=1
                    )

Found 3157 images belonging to 3 classes.


In [21]:
predictions = model.predict_generator(train_data_stream, 3157, verbose=1)
actual_classes = train_data_stream.classes
pred_classes = np.argmax(predictions, axis=1)
print ('Vanila Simple Gauss Train Accuracy Score:', accuracy_score(pred_classes, actual_classes))

3157/3157 [==============================] - 96s 30ms/step
Vanila Simple Gauss Train Accuracy Score: 0.567310738042


In [ ]:
model.load_weights('Workshop-Paper/Vanilla-Gaussian/vanila-gauss-066-0.554')

In [22]:
predictions = model.predict_generator(test_data_stream, 1979, verbose=1)
actual_classes = test_data_stream.classes
pred_classes = np.argmax(predictions, axis=1)
accuracy_score(actual_classes, pred_classes)

1979/1979 [==============================] - 67s 34ms/step


0.51490651844365842

## Training AlexNet on Raw Images

In [12]:
DROPOUT = 0.5
N_CATEGORY = 3
model_input = Input(shape = (3, 227, 227))

# First convolutional Layer (96x11x11)
z = Conv2D(filters = 96, kernel_size = (11,11), strides = (4,4), activation = "relu")(model_input)
z = MaxPooling2D(pool_size = (3,3), strides=(2,2))(z)
z = BatchNormalization()(z)

# Second convolutional Layer (256x5x5)
z = ZeroPadding2D(padding = (2,2))(z)
z = Convolution2D(filters = 256, kernel_size = (5,5), strides = (1,1), activation = "relu")(z)
z = MaxPooling2D(pool_size = (3,3), strides=(2,2))(z)
z = BatchNormalization()(z)

# Rest 3 convolutional layers
z = ZeroPadding2D(padding = (1,1))(z)
z = Convolution2D(filters = 384, kernel_size = (3,3), strides = (1,1), activation = "relu")(z)

z = ZeroPadding2D(padding = (1,1))(z)
z = Convolution2D(filters = 384, kernel_size = (3,3), strides = (1,1), activation = "relu")(z)

z = ZeroPadding2D(padding = (1,1))(z)
z = Convolution2D(filters = 256, kernel_size = (3,3), strides = (1,1), activation = "relu")(z)

z = MaxPooling2D(pool_size = (3,3), strides=(2,2))(z)
z = Flatten()(z)

z = Dense(4096, activation="relu")(z)
z = Dropout(DROPOUT)(z)

z = Dense(4096, activation="relu")(z)
z = Dropout(DROPOUT)(z)

final_dim = 1 if N_CATEGORY == 2 else N_CATEGORY
final_act = "sigmoid" if N_CATEGORY == 2 else "softmax"
model_output = Dense(final_dim, activation=final_act)(z)

model = Model(model_input, model_output)

In [21]:
WEIGHT_DECAY = 0.0005
MOMENTUM = 0.9
BATCH_SIZE = 128
LEARNING_RATE = 0.01
DROPOUT = 0.5
ALPHA = 1e-4
BETA = 0.75
n = 5
k = 2

loss_metric = "binary_crossentropy" if N_CATEGORY == 2 else "categorical_crossentropy"
model.compile(loss=loss_metric, metrics=["accuracy"], 
              optimizer=optimizers.SGD(lr=LEARNING_RATE, momentum=MOMENTUM, decay = WEIGHT_DECAY))

In [13]:
# Data augmentation
train_datagen = ImageDataGenerator(
          rotation_range=40, width_shift_range=0.2,
          height_shift_range=0.2, rescale=1./100,
          shear_range=0.2, zoom_range=0.2,
          horizontal_flip=True, fill_mode='nearest'
           )

validate_datagen = ImageDataGenerator(rescale=1./100)

In [14]:
train_data_dir = 'Data/Train/'
validation_data_dir = 'Data/Train-Val/'
nb_train_samples = 1043 + 1079 + 1145
nb_validation_samples = 116 + 120 + 127
epochs = 100
batch_size = 16

In [23]:
train_generator = train_datagen.flow_from_directory(
                    train_data_dir,
                    target_size=(227, 227),
                    batch_size=batch_size,
                    class_mode='categorical'
                    )

validation_generator = validate_datagen.flow_from_directory(
                            validation_data_dir,
                            target_size=(227, 227),
                            batch_size=batch_size,
                            class_mode='categorical'
                            )

Found 3266 images belonging to 3 classes.
Found 363 images belonging to 3 classes.


In [16]:
callback = ModelCheckpoint(filepath='Workshop-Paper/Alexnet-Raw/vanila-raw-{epoch:03d}-{val_acc:.3f}', save_best_only=True, monitor='val_acc', verbose=1)

In [ ]:
history = model.fit_generator(train_generator, callbacks=[callback],
                    steps_per_epoch=nb_train_samples // batch_size,
                    epochs=epochs,
                    validation_data=validation_generator,
                    validation_steps=nb_validation_samples // batch_size
                   )

Epoch 1/100
174/204 [========================>.....] - ETA: 23s - loss: 1.1443 - acc: 0.3373

/usr/local/lib/python3.5/dist-packages/PIL/TiffImagePlugin.py:742: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


204/204 [==============================] - 170s 834ms/step - loss: 1.1381 - acc: 0.3379 - val_loss: 1.0995 - val_acc: 0.3466
Epoch 2/100
204/204 [==============================] - 161s 791ms/step - loss: 1.0982 - acc: 0.3437 - val_loss: 1.1084 - val_acc: 0.3182
Epoch 3/100
204/204 [==============================] - 175s 858ms/step - loss: 1.1012 - acc: 0.3624 - val_loss: 1.0872 - val_acc: 0.3949
Epoch 4/100
204/204 [==============================] - 164s 802ms/step - loss: 1.0967 - acc: 0.3823 - val_loss: 1.0975 - val_acc: 0.3466
Epoch 5/100
204/204 [==============================] - 157s 770ms/step - loss: 1.0966 - acc: 0.3609 - val_loss: 1.0976 - val_acc: 0.3551
Epoch 6/100
204/204 [==============================] - 158s 773ms/step - loss: 1.0931 - acc: 0.3784 - val_loss: 1.0836 - val_acc: 0.4261
Epoch 7/100
204/204 [==============================] - 224s 1s/step - loss: 1.0935 - acc: 0.3722 - val_loss: 1.0989 - val_acc: 0.3210
Epoch 8/100
103/204 [==============>...............] - E